In this file we explore some data augmentation strategy using the phrase annotation

In [ ]:
import os
import sys
# Add project root to sys.path
cwd = os.getcwd()
dirof = os.path.dirname
project_root = dirof(dirof(cwd))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
import random
from sonata_utils import ls
from remi_z import MultiTrack
from datasets.dataset_utils import load_phrase_annot_cleaned, calculate_bar_id_of_phrases

# random.seed(42)

# data_dir = '/data1/longshen/Datasets/Piano/POP909/pop909_longshen/data_key_normed'
# song_names = ls(data_dir)

midi_fp = '/data1/longshen/Datasets/Piano/POP909/pop909_longshen/data_key_normed/001/001.mid'
mt = MultiTrack.from_midi(midi_fp)
annot_fp = '/data1/longshen/Datasets/Piano/POP909/pop909_longshen/data_key_normed/001/phrase_annot_cleaned.txt'
annot = load_phrase_annot_cleaned(annot_fp)
print(annot)


[('i', 4), ('A', 4), ('B', 8), ('A', 4), ('A', 4), ('b', 4), ('B', 8), ('A', 4), ('A', 4), ('b', 4), ('b', 4), ('A', 4), ('A', 4), ('b', 4), ('A', 4), ('o', 3)]


In [2]:
print('#Class of phrase')
phrase_type = set([x[0] for x in annot])
print(phrase_type)

#Class of phrase
{'i', 'B', 'b', 'A', 'o'}


In [3]:
calculate_bar_id_of_phrases(annot)
sections = calculate_bar_id_of_phrases(annot)
print(sections)

[('i', 0, 4), ('A', 4, 8), ('B', 8, 16), ('A', 16, 20), ('A', 20, 24), ('b', 24, 28), ('B', 28, 36), ('A', 36, 40), ('A', 40, 44), ('b', 44, 48), ('b', 48, 52), ('A', 52, 56), ('A', 56, 60), ('b', 60, 64), ('A', 64, 68), ('o', 68, 71)]


In [ ]:
# Strategy: permute that list (but maintain the first and last)
new_sections = sections[1:-1]
random.shuffle(new_sections)
new_sections = [sections[0]] + new_sections + [sections[-1]]
print(new_sections)

[('i', 0, 4), ('b', 44, 48), ('b', 24, 28), ('A', 36, 40), ('A', 40, 44), ('A', 4, 8), ('b', 48, 52), ('A', 56, 60), ('A', 52, 56), ('A', 20, 24), ('B', 28, 36), ('A', 64, 68), ('A', 16, 20), ('b', 60, 64), ('B', 8, 16), ('o', 68, 71)]


In [5]:
# Get new bar order
new_bar_order = [(x[1], x[2]) for x in new_sections]
mt_permuted = mt.permute_phrase(new_bar_order)
mt_permuted.to_midi('/home/longshen/work/AccGen/test_outputs/dataset/pop909_001_permuted.mid')


MIDI file successfully written to /home/longshen/work/AccGen/test_outputs/dataset/pop909_001_permuted.mid


In [15]:
# Strategy: permute plus duplication of phrases to maximum n_bar
# Do not keep position of first and last phrase
def augment_phrase_permute_with_duplication_keep_inout(sections, max_n_bar=128):
    middle_sections = sections[1:-1]

    # Duplication
    current_n_bar = sum([x[2]-x[1] for x in sections])
    while current_n_bar < max_n_bar:
        # Randomly select a phrase to duplicate (not the first or last)
        phrase_to_dup_idx = random.randint(1, len(sections)-2)
        phrase_to_dup = sections[phrase_to_dup_idx]
        
        len_after_dup = current_n_bar + (phrase_to_dup[2]-phrase_to_dup[1])
        if len_after_dup > max_n_bar:
            break

        middle_sections.append(phrase_to_dup)
        current_n_bar += (phrase_to_dup[2]-phrase_to_dup[1])

    random.shuffle(middle_sections)
    new_sections = [sections[0]] + middle_sections + [sections[-1]]

    return new_sections

def augment_phrase_permute_with_duplication(sections, max_n_bar=128):
    # Copy sections
    new_sections = sections.copy()

    # Duplication
    current_n_bar = sum([x[2]-x[1] for x in sections])
    while current_n_bar < max_n_bar:
        # Randomly select a phrase to duplicate (not the first or last)
        phrase_to_dup_idx = random.randint(0, len(sections)-1)
        phrase_to_dup = sections[phrase_to_dup_idx]
        
        len_after_dup = current_n_bar + (phrase_to_dup[2]-phrase_to_dup[1])
        if len_after_dup > max_n_bar:
            break

        new_sections.append(phrase_to_dup)
        current_n_bar += (phrase_to_dup[2]-phrase_to_dup[1])

    random.shuffle(new_sections)

    return new_sections

print(f'Bar Ids: {sections}')
new_bar_ids_with_dup = augment_phrase_permute_with_duplication(sections, max_n_bar=128)
print(new_bar_ids_with_dup)
print(f'Number of phrases after duplication: {len(new_bar_ids_with_dup)}')
print('Total bars:', sum([x[2]-x[1] for x in new_bar_ids_with_dup]))
mt_permuted_dup = mt.permute_phrase([(x[1], x[2]) for x in new_bar_ids_with_dup])
print(mt_permuted_dup)
mt_permuted_dup.to_midi('/home/longshen/work/AccGen/test_outputs/dataset/pop909/001_phrase_permuted_dup.mid')

Bar Ids: [('i', 0, 4), ('A', 4, 8), ('B', 8, 16), ('A', 16, 20), ('A', 20, 24), ('b', 24, 28), ('B', 28, 36), ('A', 36, 40), ('A', 40, 44), ('b', 44, 48), ('b', 48, 52), ('A', 52, 56), ('A', 56, 60), ('b', 60, 64), ('A', 64, 68), ('o', 68, 71)]
[('A', 64, 68), ('B', 8, 16), ('A', 64, 68), ('A', 4, 8), ('b', 48, 52), ('o', 68, 71), ('A', 40, 44), ('A', 16, 20), ('b', 60, 64), ('A', 4, 8), ('o', 68, 71), ('B', 28, 36), ('A', 4, 8), ('b', 48, 52), ('A', 52, 56), ('b', 44, 48), ('b', 48, 52), ('b', 48, 52), ('b', 24, 28), ('A', 36, 40), ('b', 60, 64), ('b', 60, 64), ('A', 56, 60), ('i', 0, 4), ('A', 20, 24), ('A', 16, 20), ('A', 40, 44), ('B', 8, 16), ('b', 44, 48)]
Number of phrases after duplication: 29
Total bars: 126
MultiTrack: 126 bars
MIDI file successfully written to /home/longshen/work/AccGen/test_outputs/dataset/pop909/001_phrase_permuted_dup.mid
